<a href="https://colab.research.google.com/github/ARJUN108-verma/Vector-Database/blob/main/9_Upserting_the_new_updated_files_to_pinecone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Upserting the new updated files to pinecone:-

In [ ]:
%load_ext dotenv
%dotenv

In [ ]:
!pip install pinecone

In [ ]:
import pandas as pd
import os
from pinecone import Pinecone, ServerlessSpec
from dotenv import load_dotenv, find_dotenv
import pinecone
from sentence_transformers import SentenceTransformer

In [ ]:
files = pd.read_csv("/content/course_section_descriptions.csv", encoding="cp1252")

In [ ]:
files["unique_id"] = files["course_id"].astype(str) + '-' + files["section_id"].astype(str)

In [ ]:
files["metadata"] = files.apply(lambda row: {
    "course_name": row["course_name"],
    "section_name": row["section_name"],
    "section_description": row["section_description"],
}, axis = 1)

In [ ]:
def create_embeddings(row):
    combined_text = f'''{row["course_name"]} {row["course_technology"]}
                        {row["course_description"]} {row["section_name"]}{row["section_description"]}'''
    return model.encode(combined_text, show_progress_bar = False)

In [ ]:
model = SentenceTransformer("all-MiniLM-L6-v2")

In [ ]:
files["embedding"] = files.apply(create_embeddings, axis = 1)

## Upserting data to Pinecone:-

In [ ]:
load_dotenv(find_dotenv(), override = True)

In [ ]:
pc = Pinecone(api_key = os.environ.get("PINECONE_API_KEY"), environment = os.environ.get("PINECONE_ENV"))

In [ ]:
index_name = "my-index"
dimension = 384
metric = "cosine"

In [ ]:
if index_name in [index.name for index in pc.list_indexes()]:
    pc.delete_index(index_name)
    print(f"{index_name} succesfully deleted.")
else:
     print(f"{index_name} not in index list.")

In [ ]:
pc.create_index(
    name = index_name,
    dimension = dimension,
    metric = metric,
    spec = ServerlessSpec(
        cloud = "aws",
        region = "us-east-1")
    )

In [ ]:
index = pc.Index(index_name)

In [ ]:
vectors_to_upsert = [(row["unique_id"], row["embedding"].tolist(), row["metadata"]) for index, row in files.iterrows()  ]

In [ ]:
index.upsert(vectors = vectors_to_upsert)
print("Data succesfully upserted to Pinecone index")

In [ ]:
query = "regression in Python"
query_embedding = model.encode(query, show_progress_bar=False).tolist()

In [ ]:
score_threshold = 0.4

In [ ]:
# Assuming query_results are fetched and include metadata
for match in query_results['matches']:
    if match['score'] >= score_threshold:
        course_details = match.get('metadata', {})
        course_name = course_details.get('course_name', 'N/A')
        section_name = course_details.get('section_name', 'N/A')
        section_description = course_details.get('section_description', 'No description available')

        print(f"Matched item ID: {match['id']}, Score: {match['score']}")
        print(f"Course: {course_name} \nSection: {section_name} \nDescription: {section_description}")